In [1]:
## Validando el funcionamiento de SparkContext
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1605025950554_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

In [2]:
## Carga de datos
df=spark.read.csv('s3://ngonza27datasets/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv', inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
## Visualizacion del dataset
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado| País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|  Importado|  Leve|              Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|2

In [4]:
## Creando una nueva columna
from pyspark.sql.functions import *
df2=df.withColumn("CopiedColumn", col("Edad"))
df2.show(5, False)
df.show(5, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención  |Edad|Sexo|Tipo       |Estado|País de procedencia|FIS                    |Fecha de muerte|Fecha diagnostico  |Fecha recuperado       |fecha reporte web  |CopiedColumn|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+------------+
|1         |2020-03-02 00:00:00  |11001          |Bogotá D.C.        |Bogotá D.C.             |Recuperado|19  |F   |Importado  |Leve  |Italia             |2020-02-27T00:00:00.000|- 

In [9]:
## Eliminacion de columnas
df3=df2.drop('Fecha de notificación') \
   .drop('Codigo DIVIPOLA') \
   .drop('Departamento o Distrito ') \
   .drop('fecha reporte web') \
   .drop('FIS') \
   .drop('Fecha de muerte') \
   .drop('Fecha recuperado') \
   .drop('Fecha diagnostico')

df3.show(5, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+----------+----+----+-----------+------+-------------------+------------+
|ID de caso|Ciudad de ubicación|atención  |Edad|Sexo|Tipo       |Estado|País de procedencia|CopiedColumn|
+----------+-------------------+----------+----+----+-----------+------+-------------------+------------+
|1         |Bogotá D.C.        |Recuperado|19  |F   |Importado  |Leve  |Italia             |19          |
|2         |Guadalajara de Buga|Recuperado|34  |M   |Importado  |Leve  |España             |34          |
|3         |Medellín           |Recuperado|50  |F   |Importado  |Leve  |España             |50          |
|4         |Medellín           |Recuperado|55  |M   |Relacionado|Leve  |Colombia           |55          |
|5         |Medellín           |Recuperado|25  |M   |Relacionado|Leve  |Colombia           |25          |
+----------+-------------------+----------+----+----+-----------+------+-------------------+------------+
only showing top 5 rows

In [10]:
## Filtrando datos relevantes
df3.filter(df['Ciudad de ubicación']=='Medellín') \
  .filter(df['Edad'] >= 20) \
  .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+----------+----+----+-----------+------+--------------------+------------+
|ID de caso|Ciudad de ubicación|  atención|Edad|Sexo|       Tipo|Estado| País de procedencia|CopiedColumn|
+----------+-------------------+----------+----+----+-----------+------+--------------------+------------+
|         3|           Medellín|Recuperado|  50|   F|  Importado|  Leve|              España|          50|
|         4|           Medellín|Recuperado|  55|   M|Relacionado|  Leve|            Colombia|          55|
|         5|           Medellín|Recuperado|  25|   M|Relacionado|  Leve|            Colombia|          25|
|        20|           Medellín|Recuperado|  26|   F|Relacionado|  Leve|            Colombia|          26|
|        21|           Medellín|Recuperado|  28|   M|Relacionado|  Leve|            Colombia|          28|
|        32|           Medellín|Recuperado|  55|   M|  Importado|  Leve|              España|          55|
|       106|           Medellín|Recup

In [14]:
print("Numero de ciudades en las que hay gente infectada: ", df3.select('Ciudad de ubicación').distinct().count())

## Agrupacion y consulta de datos
print("\nCiudades de Colombia con mayor numero de contagios")
df.groupBy('Ciudad de ubicación') \
    .count() \
    .orderBy('count',ascending=False) \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Numero de ciudades en las que hay gente infectada:  207

Ciudades de Colombia con mayor numero de contagios
+-------------------+-----+
|Ciudad de ubicación|count|
+-------------------+-----+
|        Bogotá D.C.| 2543|
|               Cali|  713|
|      Villavicencio|  344|
|           Medellín|  305|
|Cartagena de Indias|  246|
|       Barranquilla|  141|
|        Santa Marta|  139|
|            Pereira|  132|
|            Leticia|  104|
|              Neiva|   82|
|            Soledad|   80|
|             Cúcuta|   62|
|             Ibagué|   61|
|       Dosquebradas|   57|
|             Soacha|   57|
|            Ipiales|   56|
|              Bello|   51|
|            Palmira|   47|
|            Armenia|   45|
|         Valledupar|   43|
+-------------------+-----+
only showing top 20 rows